In [3]:
import numpy as np
from scipy.optimize import minimize_scalar
import warnings
%run Def_berekening.ipynb

#### Optimalisatie

Dit script bevat een functie die de optimalisatie van een 4-paals blokpoer uitvoert. In de optimalisatie worden de berekening en controleberekening uit het script 'berekening poer' aangeroepen. Daarnaast vraagt de functie enkele input gegevens, dit zijn:
- Paaldiameter
- Kolomdiameter
- Permanente belasting
- Variabele belasting
- Kosten beton
- Kosten staal
- Milieubelasting beton
- Milieubelasting staal
- Betonsterkteklasse

Daarnaast heeft de functie nog een aantal keyword arguments die een vaste waarde hebben, mocht het nodig zijn kunnen deze waardes ook gevarieerd worden. De breedte van de trekstang wordt berekend door een waarde bij de paaldiameter op te tellen, deze waarde is bt genoemd en is ook een keyword argument.

In de functie worden de drie optimalisatievariabelen gevarieerd. Vervolgens wordt een loop opgezet waarin een berekening wordt uitgevoerd voor alle mogelijke combinaties van variabelen. Alle poerconfiguraties die voldoen aan de toetsing van de poer worden ingevuld in een 3 dimensionale array. Mocht een poerconfiguratie niet voldoen dan wordt np.nan gereturnd. Vervolgens wordt de optimale waarde voor kosten en milieubelasting bepaald.

De output van de functie zijn de resultaten van de berekening voor de twee optimale poerconfiguraties en de waarden van de optimalisatievariabelen die daarbij horen. De functie wordt aangeroepen en gebruikt in het script 'tool'.

In [2]:
def optimalisatie(Dp, Dk, Gk, Qk, beton, kb, ks, db, ds, GammaG = 1.2, GammaQ = 1.5, staal = 'B500B', dverdeel = 12, d = 400, constructieklasse = 'S4', bt = 2 * 175, c = 50):
    b = Dp + bt           
    H = np.arange(100, 1500, 10)
    dstaaf = [12, 16, 20, 25, 32, 40]
    n = np.arange(2, 15)
    kosten = np.zeros((len(H),len(dstaaf), len(n)))
    duurzaamheid = np.zeros((len(H),len(dstaaf), len(n)))
    for i in range(len(H)):
        for j in range(len(dstaaf)):
            for k in range(len(n)):
                kosten[i,j,k] = berekening(Dp, Dk, H[i], dstaaf[j], n[k], dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c)
                duurzaamheid[i,j,k] = berekening1(Dp, Dk, H[i], dstaaf[j], n[k], dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c)
             
    if np.isnan(kosten).all():
        warnings.warn("Betonspanning is altijd hoger dan maximale betonspanning, kies hogere betonsterkteklasse of smallere kolom en/of funderingspalen", RuntimeWarning, stacklevel=3)
    
    goedkoopst = np.nanmin(kosten)
    loc = np.argwhere(kosten == goedkoopst)
   
    duurzaamst = np.nanmin(duurzaamheid)
    loc1 = np.argwhere(duurzaamheid == duurzaamst)
    
    H1 = H[loc[0][0]]
    dstaaf1 = dstaaf[loc[0][1]]
    n1 = n[loc[0][2]]
    H2 = H[loc1[0][0]]
    dstaaf2 = dstaaf[loc1[0][1]]
    n2 = n[loc1[0][2]]
    
    out1 = controleberekening(Dp, Dk, H1, dstaaf1, n1, dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c)
    out2 = controleberekening(Dp, Dk, H2, dstaaf2, n2, dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c)
    
    return out1, out2, H1, dstaaf1, n1, H2, dstaaf2, n2